# KGT COYOTE

IMPORTS


In [ ]:
import pandas as pd
import numpy as np

import os

# Directories

### Dataset have three subsets.  Train, Test, Mix 


In [ ]:
mix_dir="./COYODOG"
test_dir="./COYODOG"
train_dir="./COYODOG"


In [ ]:
test_dir_coyote=os.path.join(test_dir,"validy/coyote")
test_dir_dog=os.path.join(test_dir,"validy/dogs")

train_dir_coyote=os.path.join(test_dir,"trainy/coyote")
train_dir_dog=os.path.join(test_dir,"trainy/dogs")

In [ ]:
train_coyote_fnames = os.listdir( train_dir_coyote )
train_dog_fnames = os.listdir( train_dir_dog )
test_coyote_fnames = os.listdir( test_dir_coyote )
test_dog_fnames = os.listdir( test_dir_dog )

print(train_coyote_fnames[:5])
print(train_dog_fnames[:5])
print(test_coyote_fnames[:5])
print(test_dog_fnames[:5]) 

print("Train data Coyote: ",len(os.listdir(train_dir_coyote)))
print("Train data Dogs: ",len(os.listdir(train_dir_dog)))

print("Test Data Coyote: ",len(os.listdir(test_dir_dog)))
print("Test Data Dog: ",len(os.listdir(test_dir_coyote))) 

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Let's see the data


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

nrows = 4
ncols = 4
pic_index = 0 
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index+=4
next_coyote_pix = [os.path.join(train_dir_coyote, fname) 
                for fname in train_coyote_fnames[ pic_index-4:pic_index] 
               ]
next_dog_pix = [os.path.join(train_dir_dog, fname) 
                for fname in train_dog_fnames[ pic_index-4:pic_index]
               ]



## Loop

In [ ]:
for i, img_path in enumerate(next_coyote_pix+next_dog_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 
  img = mpimg.imread(img_path)
  plt.imshow(img)

A Small Model to start

In [ ]:
def Create_model():
  model=tf.keras.models.Sequential()
  model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(512,activation="relu"))
  model.add(layers.Dense(1,activation="sigmoid"))
  return model

In [ ]:
model=Create_model()
model.summary()

### Compile


In [ ]:
model.compile(tf.keras.optimizers.Adam(1e-4),tf.keras.losses.binary_crossentropy,metrics=['accuracy'])

# Image Processing


In [ ]:
train_datagen=ImageDataGenerator(rescale=1.0/255,rotation_range=40,
            width_shift_range=0.2,height_shift_range=0.2,
            shear_range=0.2,zoom_range=0.2,
            horizontal_flip=True,fill_mode="nearest")
test_datagen=ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_gen=train_datagen.flow_from_directory(train_dir,batch_size=20,class_mode="binary",target_size=(150,150))
validation_gen=test_datagen.flow_from_directory(test_dir,batch_size=20,class_mode="binary",target_size=(150,150))

## **Train Model**

In [ ]:
history=model.fit_generator(train_gen,epochs=1,steps_per_epoch=20,validation_data=validation_gen,validation_steps=20)

Save

In [ ]:
model.save("smallmodel.h5")

Check History

In [ ]:
history.history


Predict

In [ ]:
from tensorflow.keras.models import load_model
model=load_model("smallmodel.h5")
test_image=image.load_img("Coyotetestone.jpg",target_size=(150,150))

test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)
map=train_gen.class_indices
print(map)

if result[0][0]>0:
    predict='coyote'
else:
    predict='dog'    

print("this image is:",predict)

## Evaluating 



In [ ]:
accuracy = history.history["acc"]
loss=history.history['loss']
val_accuracy=history.history['val_acc']
val_loss=history.history['val_loss']
epochs=range(len(accuracy))

In [ ]:
plt.plot(epochs,accuracy)
plt.plot(epochs,val_accuracy)
plt.title("training and validation Accuracy")
plt.show()

In [ ]:
plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title("training and Validation loss")
plt.show()

## NOW to Create Big Model

BELOW  WE CAN CREATE A BIG MODEL NOW